# Practical Exercise 7.07: Smart Beta Portfolio (momentum) versus Index

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Define IBEX 35 Tickers
ibex35_tickers = [
    "SAN.MC", "TEF.MC", "ITX.MC", "BBVA.MC", "IBE.MC", "ACS.MC", "ANA.MC",
    "AENA.MC", "FER.MC", "CLNX.MC", "REP.MC", "ENG.MC", "RED.MC", "AMS.MC",
    "GRF.MC", "MAP.MC", "NTGY.MC", "MEL.MC", "CABK.MC", "SAB.MC",
    "COL.MC", "ELE.MC", "LOG.MC", "IAG.MC", "MRL.MC", "MTS.MC", "ACX.MC", "IDR.MC", "SCYR.MC", "ROVI.MC", "UNI.MC", "ANE.MC",
    "FDR.MC", "SLR.MC", "BKT.MC"
]

# Fetch historical price data for valid tickers
price_data = yf.download(ibex35_tickers, start="2019-01-01", end="2025-01-01", auto_adjust=False, actions=False)['Adj Close']

# Step 2: Handle Missing Data
price_data = price_data.ffill().bfill()  # Fill missing data

# Step 3: Calculate Momentum Factor
momentum = (price_data / price_data.shift(252)) - 1
momentum = momentum.iloc[-1]  # Use the most recent momentum values
momentum = momentum.clip(lower=0).dropna()  # Remove negative or invalid values
momentum_weights = momentum / momentum.sum()  # Normalize weights

# Step 4: Portfolio Construction
portfolio_weights = pd.DataFrame(index=price_data.index, columns=price_data.columns)
for ticker in price_data.columns:
    portfolio_weights[ticker] = momentum_weights.get(ticker, 0)
portfolio_weights = portfolio_weights.div(portfolio_weights.sum(axis=1), axis=0)

# Step 5: Backtesting
returns = price_data.pct_change().dropna()
portfolio_returns = (portfolio_weights.shift() * returns).sum(axis=1)
cumulative_returns = (1 + portfolio_returns).cumprod()

# Fetch and process IBEX 35 data
ibex35_index = "^IBEX"
ibex_index_data = yf.download(ibex35_index, start="2019-01-01", end="2025-01-01", auto_adjust=False, actions=False)['Adj Close']
ibex_index_data = ibex_index_data.ffill().bfill()  # Fill missing data
ibex_index_returns = ibex_index_data.pct_change().dropna()

# Calculate annualized volatility (ensure scalar)
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)
index_volatility = ibex_index_returns.std() * np.sqrt(252)

# Convert to scalar using iloc[0] if necessary
portfolio_volatility = portfolio_volatility.iloc[0] if isinstance(portfolio_volatility, pd.Series) else portfolio_volatility
index_volatility = index_volatility.iloc[0] if isinstance(index_volatility, pd.Series) else index_volatility

# Plot cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(cumulative_returns, label="Smart Beta Portfolio (Momentum)", color="green")
plt.plot((1 + ibex_index_returns).cumprod(), label="IBEX 35 Index", color="blue")
plt.title("Comparison of Momentum-Based Portfolio vs. IBEX 35 Index")
plt.xlabel("Date")
plt.ylabel("Cumulative Returns")
plt.legend()
plt.grid()
plt.show()

# Step 6: Performance Metrics
portfolio_annualized_return = (cumulative_returns.iloc[-1] ** (1 / (cumulative_returns.index[-1].year - cumulative_returns.index[0].year))) - 1
index_annualized_return = ((1 + ibex_index_returns).cumprod().iloc[-1] ** (1 / (ibex_index_returns.index[-1].year - ibex_index_returns.index[0].year))) - 1

# Convert to scalar using iloc[0] if necessary
portfolio_annualized_return = portfolio_annualized_return.iloc[0] if isinstance(portfolio_annualized_return, pd.Series) else portfolio_annualized_return
index_annualized_return = index_annualized_return.iloc[0] if isinstance(index_annualized_return, pd.Series) else index_annualized_return

# Sharpe Ratios
risk_free_rate = 0.01
portfolio_sharpe_ratio = (portfolio_annualized_return - risk_free_rate) / portfolio_volatility
index_sharpe_ratio = (index_annualized_return - risk_free_rate) / index_volatility

# Display Results
print("\nPerformance Comparison:")
print(f"Momentum Portfolio Annualized Return: {portfolio_annualized_return:.2%}")
print(f"IBEX 35 Index Annualized Return: {index_annualized_return:.2%}")
print(f"Momentum Portfolio Volatility: {portfolio_volatility:.2%}")
print(f"IBEX 35 Index Volatility: {index_volatility:.2%}")
print(f"Momentum Portfolio Sharpe Ratio: {portfolio_sharpe_ratio:.2f}")
print(f"IBEX 35 Index Sharpe Ratio: {index_sharpe_ratio:.2f}")
